In [1]:
import sys
sys.path.append('../')   # If you move this notebook to another location, change this path to point to the root directory of the solutions project
import pandas as pd
from pathlib import Path
from integrations import integration_base as base
from integrations import elc_integration as elc

# This is an optional step to make floating point numbers display more compactly.
# If you prefer other options, you can change or delete this without harm.
pd.set_option('display.float_format', lambda x: f"{x:03.5f}".rstrip('0'))

In [ ]:
# If you are beginning a *new* integration, or starting over

base.integration_clean()
base.integration_start()

# Developer Section

In [10]:
import importlib
importlib.reload(elc)

<module 'integrations.elc_integration' from 'C:\\Working\\solutions\\integrations\\..\\integrations\\elc_integration.py'>

In [2]:
df = pd.DataFrame(elc.get_emissions_factors())
df

,coal,natural gas,large hydro,oil products,onshorewind,offshorewind,solarpvutil,solarpvroof,concentratedsolar,geothermal,waveandtidal,biomass,microwind,instreamhydro,nuclear,wastetoenergy,landfillmethane,biogas
0,1028.25,574.07433,202.26478,828.22,22.4669,14.25182,46.91333,50.37818,41.94167,102.99762,20.59,112.22579,71.0485,24.6888,18.4594,1.6,226.4182,65.72984
1,1250.94861,746.91532,703.94498,1047.55152,43.91821,22.38945,74.86526,79.4291,94.02736,298.52563,36.68999,237.77484,172.09029,65.57254,45.19641,1.6,226.4182,135.43985
2,805.55139,401.23335,0.,608.88848,1.01558,6.11419,18.96141,21.32726,5.,0.,4.49001,4.,0.4,0.5,0.8,1.6,226.4182,2.43098


In [11]:
elc.gather_net_grid_use()

{'electricvehicles':            PDS1       PDS2       PDS3
 Year                                 
 2014         0.         0.         0.
 2015         0.         0.         0.
 2016         0.         0.         0.
 2017         0.         0.         0.
 2018         0.         0.         0.
 2019   19.24431   11.66632   14.46501
 2020   29.70095   59.69877   11.12599
 2021    29.0641   85.21054   41.50121
 2022   53.84925  117.20431   76.51089
 2023   82.58127  154.30408   116.0445
 2024  115.27409  196.67354  159.85398
 2025  138.94768  243.76514  207.64386
 2026   192.5979  297.87637  259.09902
 2027  237.25676  357.03712  313.73016
 2028  285.93218  422.12237   370.6247
 2029  338.63809   493.2958  429.87121
 2030  364.45854  570.87571   552.2183
 2031  456.19716  654.56204  675.21006
 2032  521.07819  744.98224  798.79105
 2033  590.04547  842.14543  921.28065
 2034  663.11293   946.2153 1045.47326
 2035  725.75361 1057.35555 1168.90746
 2036  821.60417 1175.72988 1291.25909
 2037

In [5]:
elc.testmode = True

In [3]:
from solution import factory

In [5]:
s1 = factory.load_scenario("electricvehicles")

In [9]:
s1.ua.soln_pds_net_grid_electricity_units_used()

,World,OECD90,Eastern Europe,Asia (Sans Japan),Middle East and Africa,Latin America,China,India,EU,USA
Year,,,,,,,,,,
2014,0.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015,0.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016,0.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017,0.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018,0.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019,11.66632,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020,59.69877,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021,85.21054,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022,117.20431,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
